$\newcommand{\Re}{\mathbb{R}} \newcommand{\EE}{\mathbb{E}}$
# Polynomial Optimization and the Generalized Moment Problem

## Polynomial Optimization

The polynomial optimization problem is to

$
\begin{align}
\text{minimize} \quad &f(x)\\
    \text{subject to} \quad &g_i(x) \geq 0,\quad i=1,\ldots,N
\end{align}
$

where $x \in \Re^d$ and $f(x), g_i(x) \in \Re[x]$ are polynomials. We implement the techniques developed by Parrilo et al. and Lasserre et al. where this problem is relaxed to a semidefinite program. 

In the first part of this worksheet, we show how to solve polynomial optimization problems using mompy where several example problems in the [GloptiPoly paper](http://homepages.laas.fr/henrion/papers/gloptipoly3.pdf) and the [solution extraction paper](http://homepages.laas.fr/henrion/papers/extract.pdf) are reproduced.

In the second part, we give the simplest example of estimating a mixture model by solving the Generalized Moment Problem (GMP). Some extensions can be found in [the extra examples worksheet](extra_examples.ipynb). This software was used for our paper [Estimating mixture models via mixtures of polynomials](http://papers.nips.cc/paper/5702-estimating-mixture-models-via-mixtures-of-polynomials.pdf) where we look at some more elaborate settings for the mixture model problem.

Other implementation for solving the polynomial optimization problem / and Generalized Moment Problem are [GloptiPoly](http://homepages.laas.fr/henrion/software/gloptipoly/) and [SOSTOOLS](http://www.cds.caltech.edu/sostools/) in MATLAB, and [ncpol2sdpa](https://pypi.python.org/pypi/ncpol2sdpa/) in python described [here](http://arxiv.org/abs/1308.6029).



In [1]:
# we are dependent on numpy, sympy and cvxopt.
import numpy as np
import cvxopt
import mompy as mp

# just some basic settings and setup
mp.cvxsolvers.options['show_progress'] = False
from IPython.display import display, Markdown, Math, display_markdown
sp.init_printing()

def print_problem(obj, constraints = None, moment_constraints = None):
    display_markdown(mp.problem_to_str(obj,constraints,moment_constraints, False), raw=True)

### 1) First example

In [2]:
x,y = sp.symbols('x,y')
f =  x**2 + y**2
gs = [x+y>=4, x+y<=4]
print_problem(f, gs)
sol = mp.solvers.solve_GMP(f, gs)
mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 1)

Minimizing $\mathcal{L}(x^{2} + y^{2})$

subject to 	 $x + y \geq 4$, 	 $x + y \leq 4$, 	


the maximum degree appearing in the problem is 2
slacks
1e-06
status: optimal
round=1,	 rank=1,	 size=3,	 obj=8.000
lost 0.0000001


{x: array([ 1.99999971]), y: array([ 1.99999971])}

### 2) Unconstrained optimization: the six hump camel back function

A plot of this function can be found at [library of simutations](http://www.sfu.ca/~ssurjano/camel6.html). MATLAB got the solution $x^* = [0.0898   -0.7127]$ and corresponding optimal values of $f(x^*) = -1.0316$


In [3]:
x1,x2 = sp.symbols('x1:3')
f = 4*x1**2+x1*x2-4*x2**2-2.1*x1**4+4*x2**4+x1**6/3
print_problem(f)
sol = mp.solvers.solve_GMP(f, rounds=1)
mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 2)


Minimizing $\mathcal{L}(\frac{x_{1}^{6}}{3} - 2.1 x_{1}^{4} + 4 x_{1}^{2} + x_{1} x_{2} + 4 x_{2}^{4} - 4 x_{2}^{2})$	 subject to no constraints

the maximum degree appearing in the problem is 6
slacks
1e-06
status: optimal
round=1,	 rank=3,	 size=10,	 obj=-1.032


{x1: array([ 0.08984255, -0.08984255]), x2: array([-0.71265247,  0.71265247])}

### 3) Multiple rounds

Generally more rounds are needed to get the correct solutions.

In [4]:
x1,x2,x3 = sp.symbols('x1:4')
f = -(x1 - 1)**2 - (x1 - x2)**2 - (x2 - 3)**2
gs = [1 - (x1 - 1)**2 >= 0, 1 - (x1 - x2)**2 >= 0, 1 - (x2 - 3)**2 >= 0]
print_problem(f, gs)
sol = mp.solvers.solve_GMP(f, gs, rounds=4)
mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 3)

Minimizing $\mathcal{L}(- \left(x_{1} - 1\right)^{2} - \left(x_{1} - x_{2}\right)^{2} - \left(x_{2} - 3\right)^{2})$

subject to 	 $- \left(x_{1} - 1\right)^{2} + 1 \geq 0$, 	 $- \left(x_{1} - x_{2}\right)^{2} + 1 \geq 0$, 	 $- \left(x_{2} - 3\right)^{2} + 1 \geq 0$, 	


the maximum degree appearing in the problem is 2
slacks
1e-06
status: optimal
round=1,	 rank=3,	 size=3,	 obj=-3.000
slacks
1e-06
status: optimal
round=2,	 rank=6,	 size=6,	 obj=-3.000
slacks
1e-06
status: optimal
round=3,	 rank=7,	 size=10,	 obj=-2.000
slacks
1e-06
status: optimal
round=4,	 rank=8,	 size=15,	 obj=-2.000


{x1: array([ 2.00000811,  2.00000032,  0.99999978]),
 x2: array([ 3.00000234,  2.00000006,  2.00000007])}

Yet another example

In [5]:
x1,x2,x3 = sp.symbols('x1:4')
f = -2*x1 + x2 - x3
gs = [0<=x1, x1<=2, x2>=0, x3>=0, x3<=3, x1+x2+x3<=4, 3*x2+x3<=6,\
      24-20*x1+9*x2-13*x3+4*x1**2 - 4*x1*x2+4*x1*x3+2*x2**2-2*x2*x3+2*x3**2>=0]; hs = [];
print_problem(f, gs)
sol = mp.solvers.solve_GMP(f, gs, hs, rounds=4)
print mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 2)

Minimizing $\mathcal{L}(- 2 x_{1} + x_{2} - x_{3})$

subject to 	 $x_{1} \geq 0$, 	 $x_{1} \leq 2$, 	 $x_{2} \geq 0$, 	 $x_{3} \geq 0$, 	 $x_{3} \leq 3$, 	 $x_{1} + x_{2} + x_{3} \leq 4$, 	 $3 x_{2} + x_{3} \leq 6$, 	 $4 x_{1}^{2} - 4 x_{1} x_{2} + 4 x_{1} x_{3} - 20 x_{1} + 2 x_{2}^{2} - 2 x_{2} x_{3} + 9 x_{2} + 2 x_{3}^{2} - 13 x_{3} + 24 \geq 0$, 	


the maximum degree appearing in the problem is 2
slacks
1e-06
status: optimal
round=1,	 rank=4,	 size=4,	 obj=-6.000
slacks
1e-06
status: optimal
round=2,	 rank=8,	 size=10,	 obj=-5.692
slacks
1e-06
status: optimal
round=3,	 rank=15,	 size=20,	 obj=-4.068
slacks
1e-06
status: optimal
round=4,	 rank=17,	 size=35,	 obj=-4.000
{x3: array([  3.48007643e-07,   2.99999775e+00]), x1: array([ 1.99999986,  0.50000288]), x2: array([ -1.10400714e-08,   1.46357718e-05])}


### 4) Motzkin polynomial
The Motzkin polynomial is non-negative, but cannot be expressed in sum of squares. It attains global minimum of 0 at $|x_1| = |x_2| = \sqrt{3}/3$ (4 points). The first few relaxations are unbounded and might take a while for cvxopt to realize this.

In [6]:
x1,x2 = sp.symbols('x1:3')
f = x1**2 * x2**2 * (x1**2 + x2**2 - 1) + 1./27
print_problem(f)
sol = mp.solvers.solve_GMP(f, rounds=7)


Minimizing $\mathcal{L}(x_{1}^{2} x_{2}^{2} \left(x_{1}^{2} + x_{2}^{2} - 1\right) + 0.037037037037037)$	 subject to no constraints

the maximum degree appearing in the problem is 6
slacks
1e-06
status: unknown
round=1,	 rank=10,	 size=10,	 obj=-1119264077.823
slacks
1e-06
status: unknown
round=2,	 rank=15,	 size=15,	 obj=-408995.200
slacks
1e-06
status: unknown
round=3,	 rank=21,	 size=21,	 obj=-5690.864
slacks
1e-06
status: unknown
round=4,	 rank=28,	 size=28,	 obj=-918.009
slacks
1e-06
status: unknown
round=5,	 rank=36,	 size=36,	 obj=-67.576
slacks
1e-06
status: optimal
round=6,	 rank=13,	 size=45,	 obj=0.000
slacks
1e-06
status: optimal
round=7,	 rank=16,	 size=55,	 obj=0.000


In [7]:
print mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 4, maxdeg=3)

{x1: array([-0.57735018, -0.5773504 ,  0.57735014,  0.57735044]), x2: array([ 0.57735087, -0.57735087, -0.57735112,  0.57735112])}


## Generalized Moment Problem (GMP)
The GMP is to

$
\begin{align}
\text{minimize} \quad &f(x)\\
    \text{subject to} \quad &g_i(x) \geq 0,\quad i=1,\ldots,N\\
    \quad &\mathcal{L}(h_j(x)) \geq 0,\quad j=1,\ldots,M
\end{align}
$

where $x \in \Re^d$ and $f(x), g_i(x), h_j(x) \in \Re[x]$ are polynomials, and $\mathcal{L}(\cdot): \Re[x] \mapsto \Re$ is a linear functional. Loosely speaking, we would like a measure $\mu$ to exist so that $\mathcal{L}(h) = \int h\ d\mu$.


For details see:


Jean B. Lasserre, 2011, Moments, Positive Polynomials and Their Applications

Jean B. Lasserre, 2008, A semidefinite programming approach to the generalized problem of moments
 

### Estimating mixtures of exponential distributions
We are interested in estimating the parameters of a mixture model from some observed moments, possibly with constraints on the parameters. Suppose that we have a mixture of 2 exponential distributions with density function $p(x; \beta) \sim \exp(-x/\beta)/\beta$ in for $x \in [0,\infty)$. Suppose that $\pi_1 = \pi_2 = 0.5$ and $\beta_1 = 1, \beta_2 = 2$. We are allowed to observe moments of the data $\EE[X^n] = {n!}(\pi_1 \beta_1^n + \pi_2 \beta_2^n)$ for $n=1,\ldots,6$. For more details on estimating mixture model using this method, see our paper [Estimating mixture models via mixtures of polynomials](http://papers.nips.cc/paper/5702-estimating-mixture-models-via-mixtures-of-polynomials.pdf) and see [the extra examples worksheet](extra_examples.ipynb) for another easy example on mixture of Gaussians.

In [8]:
beta = sp.symbols('beta')
beta0 = [1,2]; pi0 = [0.5,0.5]
hs = [beta**m - (pi0[0]*beta0[0]**m+pi0[1]*beta0[1]**m) for m in range(1,5)]
f = sum([beta**(2*i) for i in range(3)])
# note that hs are the LHS of h==0, whereas gs are sympy inequalities
print_problem(f, None, hs)

sol = mp.solvers.solve_GMP(f, None, hs)
print mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 2, tol = 1e-3)

Minimizing $\mathcal{L}(\beta^{4} + \beta^{2} + 1)$

subject to 	$\mathcal{L}(\beta - 1.5) = 0$, 	$\mathcal{L}(\beta^{2} - 2.5) = 0$, 	$\mathcal{L}(\beta^{3} - 4.5) = 0$, 	$\mathcal{L}(\beta^{4} - 8.5) = 0$, 	

the maximum degree appearing in the problem is 4
slacks
1e-06
status: optimal
round=1,	 rank=2,	 size=3,	 obj=12.000
lost 0.0000000
{beta: array([ 1.,  2.])}


Now we try to solve the problem with insufficient moment conditions, but extra constraints on the parameters themselves.

In [9]:
gs=[beta>=1]
f = sum([beta**(2*i) for i in range(3)])
hs_sub = hs[0:2]
print_problem(f, gs, hs_sub)

sol = mp.solvers.solve_GMP(f, gs, hs_sub)
print mp.extractors.extract_solutions_lasserre(sol['MM'], sol['x'], 2, tol = 1e-3)

Minimizing $\mathcal{L}(\beta^{4} + \beta^{2} + 1)$

subject to 	 $\beta \geq 1$, 	
$\mathcal{L}(\beta - 1.5) = 0$, 	$\mathcal{L}(\beta^{2} - 2.5) = 0$, 	

the maximum degree appearing in the problem is 4
slacks
1e-06
status: optimal
round=1,	 rank=2,	 size=3,	 obj=12.000
lost 0.0000001
{beta: array([ 0.99999876,  1.99999896])}
